In [ ]:
import keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
#from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception

# Define image size
img_width, img_height = 224, 224

# Define dataset directories
train_data_dir = r"C:\Users\umasr\Downloads\Normal"
validation_data_dir = r"C:\Users\umasr\Downloads\pneumonia"

# Define batch size
batch_size = 32

# Define number of epochs
epochs = 10

# Define number of classes
num_classes = 2

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load pre-trained model and exclude top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add new top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define new model with top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height),
                                                        batch_size=batch_size, class_mode='categorical')
model.fit_generator(train_generator, epochs=epochs, steps_per_epoch=len(train_generator),
                    validation_data=validation_generator, validation_steps=len(validation_generator))

# Save model
model.save('pneumonia_detection.h5')
